In [1]:
from utils import get_train_test_datasets
import pandas as pd
from autogluon.tabular import TabularPredictor
import pandas as pd
import json
import h5py
import pickle
import numpy as np

/home/julian/miniconda3/envs/dtiam-esm/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
included_model_types = ['NN_TORCH', 'FASTAI', 'GBM', 'XGB']
hyperparameters = {
	'NN_TORCH': [{'ag_args_fit': {'num_gpus': 1}}],
	'GBM': [
        {},
        {'learning_rate': 0.03, 'num_leaves': 128, 'feature_fraction': 0.9, 'min_data_in_leaf': 3, 'ag_args': {'name_suffix': 'Large', 'priority': 0, 'hyperparameter_tune_kwargs': None}}],
	'XGB': [{'ag_args_fit': {'num_gpus': 1}}],
	'FASTAI': [{'ag_args_fit': {'num_gpus': 1}}]
}
eval_metric = 'roc_auc'
preset = None
problem_type = 'binary'

In [3]:
# dataset = 'hetionet'
dataset = 'yamanishi_08'
setting = 'warm_start'
# setting = 'drug_coldstart'
# setting = 'protein_coldstart'
target = 'dti'
fold = 0
mol_model = 'MolE_GuacaMol_27113.ckpt'
# mol_model = 'BerMolModel_base.pkl'
# mol_model = 'dtiam_kiba_compound_features'
train_df, test_df = get_train_test_datasets(fold=fold, setting=setting, dataset=dataset, mol_model=mol_model, target=target)

Index(['drug_id', 'protein_id', 'affinity'], dtype='object')


In [4]:
train_df.shape

(50757, 2049)

In [5]:
for fold in range(5):
    dataset_path = "../data/dta/" + dataset
    folds_path = dataset_path + f"/data_folds/{setting}/"

    train_df, test_df = get_train_test_datasets(fold=fold, setting=setting, dataset=dataset, mol_model=mol_model, target=target)
    
    model_path = f"/home/julian/DTIAM/code/AutogluonModels/{dataset}-{setting}-fold_{fold}-{mol_model.split('.')[0]}"
    print(model_path)

    predictor = TabularPredictor(path=model_path, label="y", eval_metric=eval_metric, problem_type=problem_type).fit(
        train_data=train_df, included_model_types=included_model_types, presets=preset, hyperparameters=hyperparameters
    )

Index(['drug_id', 'protein_id', 'affinity'], dtype='object')


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.10.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #26~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Apr 17 19:20:47 UTC 2
CPU Count:          20
Memory Avail:       44.04 GB / 62.63 GB (70.3%)
Disk Space Avail:   163.56 GB / 1684.96 GB (9.7%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.
	presets='high'         : Strong accuracy with f

/home/julian/DTIAM/code/AutogluonModels/yamanishi_08-warm_start-fold_0-MolE_GuacaMol_27113


Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    45046.63 MB
	Train Data (Original)  Memory Usage: 396.54 MB (0.9% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 2048 | ['0', '1', '2', '3', '4', ...]
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 2048 | ['0', '1', '2', '3', '4', ...]
	3.4s = Fit runtime
	2048 features in ori

[1000]	valid_set's binary_logloss: 0.0891715


...

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.10.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #26~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Apr 17 19:20:47 UTC 2
CPU Count:          20
Memory Avail:       33.36 GB / 62.63 GB (53.3%)
Disk Space Avail:   161.88 GB / 1684.96 GB (9.6%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.
	presets='high'         : Strong accuracy with f

/home/julian/DTIAM/code/AutogluonModels/yamanishi_08-warm_start-fold_4-MolE_GuacaMol_27113


Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    34500.12 MB
	Train Data (Original)  Memory Usage: 396.54 MB (1.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 2048 | ['0', '1', '2', '3', '4', ...]
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 2048 | ['0', '1', '2', '3', '4', ...]
	3.3s = Fit runtime
	2048 features in ori

[1000]	valid_set's binary_logloss: 0.0957985


	0.9653	 = Validation score   (roc_auc)
	79.19s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
No improvement since epoch 8: early stopping
	0.978	 = Validation score   (roc_auc)
	43.05s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: XGBoost ...
	0.9695	 = Validation score   (roc_auc)
	15.75s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.9886	 = Validation score   (roc_auc)
	163.3s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: LightGBMLarge ...
	0.969	 = Validation score   (roc_auc)
	109.75s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'NeuralNetTorch': 1.0}
	0.9886	 = Validation score   (roc_auc)
	0.03s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 416.37s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 47868.6 rows/s (2500 batch size

In [ ]:
# ag-20250616_230514 es drug_coldstart

In [16]:
# predictor.path = mol_model

In [8]:
test_data_nolab = test_df.drop(columns=["y"])

In [9]:
pred_scores = predictor.predict(test_data_nolab)

In [10]:
import numpy as np
from utils import load_data, rmse, mse, pearson, spearman, ci, roc_auc, pr_auc
res_all = pd.DataFrame(columns=["RMSE", "MSE", "Pearson", "Spearman", "CI"])
G, P = np.array(test_df["y"]), np.array(pred_scores)
ret = [rmse(G, P), mse(G, P), pearson(G, P), spearman(G, P), ci(G, P)]

In [11]:
i = 0
print(
    f"fold: {i+1}, RMSE: {ret[0]}, MSE: {ret[1]}, Pearson: {ret[2]}, Spearman: {ret[3]}, CI: {ret[4]}"
)
res_all.loc[i] = ret

fold: 1, RMSE: 0.5849176988608807, MSE: 0.342128714440708, Pearson: 0.7199550158109927, Spearman: 0.638433401990211, CI: 0.7514334587237242
